<a href="https://colab.research.google.com/github/VaidehiMagare/AI-CHATBOT-WITH-NLP/blob/main/AI_CHATBOT_WITH_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
shutil.rmtree('/usr/local/nltk_data', ignore_errors=True)
shutil.rmtree('/root/nltk_data', ignore_errors=True)

In [2]:
import random
import string  # to remove punctuation

In [3]:
# ✅ Installing and loading spaCy
!pip install -q spacy
!python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
import requests

In [21]:
NEWS_API_KEY = "cca7a2a355264b7cb72588f73b2cd23e"

url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={NEWS_API_KEY}"

response = requests.get(url)
data = response.json()
print(data["status"])
print(len(data.get("articles", [])))

ok
17


In [22]:
NEWS_API_KEY = "cca7a2a355264b7cb72588f73b2cd23e"

def get_top_news():
    print("DEBUG: Inside get_top_news()")
    urls = [
        f"https://newsapi.org/v2/top-headlines?country=in&apiKey={NEWS_API_KEY}",
        f"https://newsapi.org/v2/top-headlines?country=us&apiKey={NEWS_API_KEY}"
    ]

    for url in urls:
        try:
            response = requests.get(url)
            data = response.json()
            if data["status"] == "ok":
                articles = data.get("articles", [])[:5]
                if articles:
                    return "\n".join([f"{i+1}. {a['title']}" for i, a in enumerate(articles)])
        except:
            continue

    return "✅ News API is working, but no headlines are available at the moment. Try again later!"

In [23]:
# knowledge base

knowledge_base = {
    "greeting": [
        "hello", "hi", "hii", "hey", "good morning", "good evening", "what's up", "yo", "howdy"
    ],
    "goodbye": [
        "bye", "goodbye", "see you", "take care", "later", "see ya", "talk to you later"
    ],
    "how_are_you": [
        "how are you", "how's it going", "how are you doing", "what's up with you", "are you okay"
    ],
    "thanks": [
        "thanks", "thank you", "much appreciated", "thx", "thanks a lot", "thank you very much"
    ],
    "about": [
        "who are you", "what can you do", "tell me about yourself", "what are you", "describe yourself"
    ],
    "news": ["news", "tell me the news", "headlines", "latest news"] ,

    "help": [
        "help", "can you help me", "i need assistance", "i need help", "support me"
    ],
    "name": [
        "what is your name", "who are you", "do you have a name", "tell me your name"
    ],
    "weather": [
        "what's the weather", "is it raining", "how's the weather", "weather forecast", "will it rain today"
    ],
    "joke": [
        "tell me a joke", "make me laugh", "say something funny", "i want to hear a joke"
    ],
    "age": [
        "how old are you", "what's your age", "when were you created", "your age"
    ]
}


responses = {
    "greeting": "Hello! How can I assist you today?",
    "goodbye": "Goodbye! Have a nice day.",
    "how_are_you": "I'm a chatbot, always ready to help!",
    "thanks": "You're welcome! Happy to assist.",
    "about": "I'm a simple chatbot created using SpaCy. I can answer basic questions and chat with you.",
    "help": "Sure, I'm here to help! Ask me anything or tell me what you need.",
    "name": "You can call me ChatBot! I'm your virtual assistant.",
    "weather": "I can't check live weather yet, but I suggest looking at a weather website or app!",
    "joke": "Why don’t scientists trust atoms? Because they make up everything! 😄",
    "age": "I'm timeless! But I was created recently to assist you with basic tasks and fun chats.",
    "news": get_top_news  ,  # NOTE: this is a callable now, not a string
}

In [24]:
def preprocess(text):
  # convert to lowercase - makes matching consitent
  doc = nlp(text.lower())

  # Return the final list of cleaned, meaningful words
  return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

In [29]:
# takes user input (a string) and returns an appropriate chatbot response
def get_responses(user_input):
    user_tokens = set(preprocess(user_input))

    if "news" in user_tokens:
        return get_top_news()

    cleaned_text = " ".join(preprocess(user_input))
    best_intent = None
    best_score = 0

    best_intent = None
    best_score = 0

    for intent, keyword_phrases in knowledge_base.items():
        for phrase in keyword_phrases:
            phrase_tokens = set(preprocess(phrase))
            match_score = len(user_tokens & phrase_tokens)
            if match_score > best_score:
                best_score = match_score
                best_intent = intent

    if best_intent:
        response = responses[best_intent]
        return response() if callable(response) else response

    return "I'm sorry, I don't understand that. Can you rephrase?"

In [30]:
def chatbot():
  print("Bot: Hi! Type 'exit' to end the conversation :)")

  while True:
    user_input = input("You: ")

    if user_input in ['exit', 'quit', 'Exit', 'Quit']:
      print("Bot: Goodbye! 👋 Feel free to come back anytime if you have more questions or need help. Take care! 😊")
      break

    response = get_responses(user_input)
    print("Bot: ", response)

In [31]:
preprocess("What's the best way to learn coding?")

['good', 'way', 'learn', 'coding']

In [32]:
chatbot()

Bot: Hi! Type 'exit' to end the conversation :)
You: hi
Bot:  Hello! How can I assist you today?
You: news
DEBUG: Inside get_top_news()
Bot:  1. PayPal teams up with the Big Ten and Big 12 to enable payments to student-athletes - CNBC
2. Newly single Orlando Bloom warmly greets Kim Kardashian in Venice after that viral photo checking her out - Page Six
3. Iran's supreme leader claims 'victory' over Israel before first video message since ceasefire - BBC
4. Foldable iPhone Leak Reveals New Camera Details - MacRumors
5. 'The Bear' is back — and leaning into its strengths in Season 4 - NPR
You: tell me a joke
Bot:  Why don’t scientists trust atoms? Because they make up everything! 😄
You: what is your age
Bot:  I'm timeless! But I was created recently to assist you with basic tasks and fun chats.
You: what all can you do
Bot:  I'm sorry, I don't understand that. Can you rephrase?
You: bye
Bot:  Goodbye! Have a nice day.
You: quit
Bot: Goodbye! 👋 Feel free to come back anytime if you have m